In [1]:
import pandas as pd
import json
file_path = '/home/fjiriges/CodeXGLUE/Code-Code/Defect-detection/dataset/test.jsonl'

examples = [] 
labels = []
with open(file_path) as f:
            for line in f:
                js=json.loads(line.strip())
                examples.append(' '.join(js['func'].split()))
                labels.append(js['target'])

In [2]:
index = 6
code_example = examples[index]
print(code_example)
label = labels[index]
print(label)

static void scsi_read_request(SCSIDiskReq *r) { SCSIDiskState *s = DO_UPCAST(SCSIDiskState, qdev, r->req.dev); uint32_t n; if (r->sector_count == (uint32_t)-1) { DPRINTF("Read buf_len=%zd\n", r->iov.iov_len); r->sector_count = 0; scsi_req_data(&r->req, r->iov.iov_len); return; } DPRINTF("Read sector_count=%d\n", r->sector_count); if (r->sector_count == 0) { scsi_command_complete(r, GOOD, NO_SENSE); return; } /* No data transfer may already be in progress */ assert(r->req.aiocb == NULL); n = r->sector_count; if (n > SCSI_DMA_BUF_SIZE / 512) n = SCSI_DMA_BUF_SIZE / 512; r->iov.iov_len = n * 512; qemu_iovec_init_external(&r->qiov, &r->iov, 1); r->req.aiocb = bdrv_aio_readv(s->bs, r->sector, &r->qiov, n, scsi_read_complete, r); if (r->req.aiocb == NULL) { scsi_read_complete(r, -EIO); } }
1


In [3]:
import torch
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          BertConfig, BertForMaskedLM, BertTokenizer,
                          GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
                          OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,
                          DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)

We load pre-trained model and tokenizers

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_path = "microsoft/codebert-base"

#load model
model = RobertaForSequenceClassification.from_pretrained(model_path, output_attentions=True)
model.to(device)
model.eval()
model.zero_grad()
# load tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_path)

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be 

In [ ]:
tokenized_text = tokenizer.tokenize(code_example)

In [35]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerActivation
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [41]:
# A helper function to perform fowad pass and get the attention weights

def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, 
                   token_type_ids=token_type_ids,
                   position_ids = position_ids,
                   attention_mask = attention_mask)
    return output.logits, output.attentions

In [12]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [13]:
ref_token_id, sep_token_id, cls_token_id

(1, 2, 0)

In [19]:
code_ids = tokenizer.encode(code_example, add_special_tokens=False)

Below we define a set of helper function for constructing references / baselines for word tokens, token types and position ids.

In [14]:
def construct_input_ref_pair(code_snippet, ref_token_id, sep_token_id, cls_token_id):
    code_ids = tokenizer.encode(code_snippet, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + code_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(code_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(code_ids)


In [18]:
def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

In [22]:
ids = construct_input_ref_pair(code_example, ref_token_id, sep_token_id, cls_token_id)

In [23]:
res = construct_input_ref_token_type_pair(ids[0])

In [27]:
def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

In [32]:
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [33]:
def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = interpretable_embedding.indices_to_embeddings(input_ids)
    ref_input_embeddings = interpretable_embedding.indices_to_embeddings(ref_input_ids)
    
    return input_embeddings, ref_input_embeddings

In [36]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(code_example, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [ ]:
logits = model(input_ids)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.